In [ ]:
!pip install nltk

In [44]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
from nltk.translate.bleu_score import sentence_bleu


In [45]:
# Load the pre-trained model and tokenizer
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [46]:
def calculate_bleu_score(prediction, ground_truth):
    # Tokenize the prediction and ground truth for BLEU scoring
    pred_tokens = prediction.split()
    gt_tokens = ground_truth.split()
    
    # Reference is a list of lists of tokens
    references = [gt_tokens]
    
    # Calculate BLEU score
    score = sentence_bleu(references, pred_tokens)
    return score

In [36]:
def answer_question(question, context, ground_truth=None):
    # Tokenize input question and context
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")

    # Get input ids and attention mask
    input_ids = inputs["input_ids"].tolist()[0]

    # Get the model's output
    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Get the most likely beginning and end of answer with the argmax of the score
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    # Convert tokens to the answer
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    # Calculate BLEU score
    if ground_truth:
        bleu_score = calculate_bleu_score(answer, ground_truth)
        return answer, bleu_score
    else:
        return answer

In [47]:
# Define a context and a question
context = """
The Transformers library by Hugging Face provides general-purpose architectures for Natural Language Understanding (NLU) and Natural Language Generation (NLG) with pretrained models in over 100 languages.
These models can be used on a variety of tasks, including text classification, information extraction, question answering, summarization, translation, and text generation.
"""

**Direct Prompting**:

In [57]:
context = """
The Transformers library by Hugging Face provides general-purpose architectures for Natural Language Understanding (NLU) and Natural Language Generation (NLG) with pretrained models in over 100 languages.
These models can be used on a variety of tasks, including text classification, information extraction, question answering, summarization, translation, and text generation.
"""
question = "What does the Transformers library provide?"
ground_truth = "general-purpose architectures for Natural Language Understanding and Natural Language Generation"

# Get the answer and BLEU score
# Get the answer
answer,bleu_score = answer_question(question, context,ground_truth)
print(f"Question: {question}")
print(f"Answer: {answer}")
print(f'BLEU_SCORE:{bleu_score}')

Question: What does the Transformers library provide?
Answer: general - purpose architectures for natural language understanding ( nlu ) and natural language generation ( nlg )
BLEU_SCORE:0


**Zero-shot Prompting**

In [58]:
context = """
The Transformers library by Hugging Face provides general-purpose architectures for Natural Language Understanding (NLU) and Natural Language Generation (NLG) with pretrained models in over 100 languages.
These models can be used on a variety of tasks, including text classification, information extraction, question answering, summarization, translation, and text generation.
"""
question = "What are the applications of transformers library?"
ground_truth ='text classification, information extraction, question answering, summarization, translation, and text generation'
# Get the answer
answer,bleu_score = answer_question(question, context,ground_truth)
print(f"Question: {question}")
print(f"Answer: {answer}")
print(f'BLEU_SCORE:{bleu_score}')

Question: What are the applications of transformers library?
Answer: text classification, information extraction, question answering, summarization, translation, and text generation
BLEU_SCORE:1


**Few-shot prompting**

In [52]:
context = 'Transformers library by Hugging Face provides general-purpose architectures for Natural Language Understanding (NLU) and Natural Language Generation (NLG) with pretrained models in over 100 languages.'
question = 'what does the Transformers library provide?'
ground_truth = 'general-purpose architectures'
answer = answer_question(question, context,ground_truth)
print(f"Question: {question}")
print(f"Answer: {answer}")
print(f'BLEU_SCORE:{bleu_score}')

Question: what does the Transformers library provide?
Answer: ('general - purpose architectures for natural language understanding', 0)
BLEU_SCORE:1


**Cloze-prompting**

In [61]:
context = """
The Transformers library by Hugging Face provides general-purpose architectures for Natural Language Understanding (NLU) and Natural Language Generation (NLG) with pretrained models in over 100 languages.
These models can be used on a variety of tasks, including text classification, information extraction, question answering, summarization, translation, and text generation.
"""
question = "The Transformers library by hugging face provides _____ for Natural Language Processing."
ground_truth = 'general-purpose architectures'
answer = answer_question(question, context,ground_truth)
print(f"Question: {question}")
print(f"Answer: {answer}")
print(f'BLEU_SCORE:{bleu_score}')

Question: The Transformers library by hugging face provides _____ for Natural Language Processing.
Answer: ('general - purpose architectures', 0)
BLEU_SCORE:1


**Instructional Prompting**

In [62]:
context = """
The Transformers library by Hugging Face provides general-purpose architectures for Natural Language Understanding (NLU) and Natural Language Generation (NLG) with pretrained models in over 100 languages.
These models can be used on a variety of tasks, including text classification, information extraction, question answering, summarization, translation, and text generation.
"""
question = "List the key features provided by the Transformers library."
ground_truth = 'text classification, information extraction, question answering, summarization, translation, and text generation'
answer = answer_question(question, context,ground_truth)
print(f"Question: {question}")
print(f"Answer: {answer}")
print(f'BLEU_SCORE:{bleu_score}')


Question: List the key features provided by the Transformers library.
Answer: ('text classification, information extraction, question answering, summarization, translation, and text generation', 1)
BLEU_SCORE:1


**Chain-of-thought-prompting**

In [64]:
context = """
The Transformers library by Hugging Face provides general-purpose architectures for Natural Language Understanding (NLU) and Natural Language Generation (NLG) with pretrained models in over 100 languages.
These models can be used on a variety of tasks, including text classification, information extraction, question answering, summarization, translation, and text generation.
"""
question = "what tasks can the Transformers library handle? How does it accomplish these tasks?"
ground_truth = '''for Natural Language Understanding (NLU) and Natural Language Generation (NLG) with pretrained models in over 100 languages.
These models can be used on a variety of tasks, including text classification, information extraction, question answering, summarization, translation, and text generation.'''
answer = answer_question(question, context,ground_truth)
print(f"Question: {question}")
print(f"Answer: {answer}")
print(f'BLEU_SCORE:{bleu_score}')

Question: what tasks can the Transformers library handle? How does it accomplish these tasks?
Answer: ('text classification, information extraction, question answering, summarization, translation, and text generation', 0)
BLEU_SCORE:1
